In [1]:
import json
import pandas as pd 
import numpy as np

In [9]:
%load_ext autoreload
%autoreload 2

import anndata
import matplotlib.pyplot as plt
import seaborn as sns
import logging
import numpy as np
import pandas as pd
import scipy.stats
import scanpy as sc
import batchglm.api as glm
import diffxpy.api as de

print("batchglm version "+glm.__version__)
print("diffpy version "+de.__version__)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


2023-07-14 17:32:23.576043: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-14 17:32:23.577152: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-14 17:32:23.600719: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-14 17:32:23.601122: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-14 17:32:24.078354: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

batchglm version v0.7.4
diffpy version v0.7.4


In [2]:
import scipy
from statsmodels.stats.multitest import fdrcorrection

In [22]:
labels = pd.read_csv("df_new_temp.csv")
my_file = open("DA_regions.txt", "r")
data = my_file.read()
da_regions = data.split("\n")
my_file.close()
flat_list = []
for item in da_regions:
    try:
        item = item.split(']')[1].strip()  # Remove the part before ']'
        flat_list.extend(item.split())  # Split and extend the flat_list
    except:
        break
        
labels = labels.drop("Unnamed: 0", axis=1)
#labels = labels.set_index("label")
labels["da_regions"] = flat_list
labels["da_regions"] = labels["da_regions"].astype(int)
#labels["condition"] = np.where(labels['label'].str.contains("ALS"), "Case", "Control")
labels["condition"] = np.where(labels['label'].str.contains("ALS"), 1, 0)

labels["sample"] = labels["label"].str[:-12]

In [28]:
adata = sc.AnnData(X=labels.iloc[:,:-4].to_numpy(), obs=labels[["condition", "da_regions"]])

/data/je30bery/.local/lib/python3.9/site-packages/anndata/_core/anndata.py:117: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [30]:
adata

AnnData object with n_obs × n_vars = 171154 × 19
    obs: 'condition', 'da_regions'

In [27]:
test = de.test.pairwise(
    data=adata,
    grouping="da_regions",
    test="z-test",
    lazy=False,
    noise_model="nb"
)

ValueError: type <class 'tuple'> not recognized

In [15]:
labels["da_regions"]

0         0
1         0
2         0
3         0
4         0
         ..
171149    0
171150    0
171151    0
171152    0
171153    0
Name: da_regions, Length: 171154, dtype: int64

In [4]:
labels

,CD11b-PE,CD16-PE,CD2-FITC,CD26-PE,CD29-FITC,CD3-PE,CD36-FITC,CD38-PE,CD4-PE,CD45-PE,...,CD7-PE,CD8-PE,HLA-DQ-FITC,HLA-DR-PE,NeuN-AF488,PBS,label,da_regions,condition,sample
0,0.000000,0.000000,0.015464,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090206,...,0.000000,0.000000,0.492268,0.502577,0.0,0.023196,Healthy01 - 21315.pkl,0,Control,Healthy01
1,0.000000,0.000000,0.200737,0.871087,0.084715,0.489871,0.000000,0.000000,0.493554,0.322284,...,0.000000,0.000000,0.069982,0.001842,0.0,0.000000,Healthy01 - 21315.pkl,0,Control,Healthy01
2,0.000000,0.000000,0.192440,0.000000,0.113402,0.109966,0.000000,0.000000,0.000000,0.000000,...,0.030928,0.398625,0.065292,0.000000,0.0,0.000000,Healthy01 - 21315.pkl,0,Control,Healthy01
3,0.000000,0.000000,0.443750,0.068750,0.106250,0.731250,0.025000,0.000000,0.856250,0.281250,...,0.381250,0.000000,0.043750,0.000000,0.0,0.000000,Healthy01 - 21315.pkl,0,Control,Healthy01
4,0.000000,0.000000,0.136656,0.254019,0.000000,0.310289,0.000000,0.000000,0.297428,0.347267,...,0.213826,0.000000,0.360129,0.000000,0.0,0.000000,Healthy01 - 21315.pkl,0,Control,Healthy01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171149,0.000000,0.118421,0.019737,0.000000,0.118421,0.000000,0.138158,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.072368,0.006579,0.0,0.000000,ALS03 - 21308.pkl,0,Case,ALS03
171150,0.000000,0.000000,0.056291,0.000000,0.000000,0.811258,0.000000,0.000000,0.685430,0.036424,...,0.278146,0.000000,0.119205,0.000000,0.0,0.000000,ALS03 - 21308.pkl,0,Case,ALS03
171151,0.305439,0.000000,0.108787,0.000000,0.000000,0.083682,0.000000,0.142259,0.000000,0.171548,...,0.439331,0.589958,0.263598,0.029289,0.0,0.000000,ALS03 - 21308.pkl,0,Case,ALS03
171152,0.000000,0.000000,0.019455,0.000000,0.000000,0.272374,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.412451,0.031128,0.000000,0.0,0.000000,ALS03 - 21308.pkl,0,Case,ALS03


In [5]:
crash

NameError: name 'crash' is not defined

In [ ]:
import seaborn as sns 
import matplotlib.pyplot as plt

In [ ]:
labels["da_regions"]

In [ ]:
#plt.clf()
sns.displot(labels, x="da_regions", hue="condition", kind="hist", multiple="stack", bins=len(np.unique(labels["da_regions"].values)))
plt.yscale("log")

In [ ]:
#plt.clf()
sns.displot(labels, x="da_regions", hue="label", kind="hist", multiple="stack", bins=len(np.unique(labels["da_regions"].values)))
plt.yscale("log")

In [ ]:
#plt.clf()
#labels = labels[labels["da_regions"] != 0]
sns.displot(labels[labels["condition"] == "Case"] , x="da_regions", hue="sample", kind="hist", multiple="stack", bins=len(np.unique(labels["da_regions"].values)))
#plt.yscale("log")
plt.xticks(range(len(np.unique(labels["da_regions"].values))))
plt.show()

In [ ]:
#plt.clf()
#labels = labels[labels["da_regions"] != 0]
sns.displot(labels[labels["condition"] == "Control"] , x="da_regions", hue="sample", kind="hist", multiple="stack", bins=len(np.unique(labels["da_regions"].values)))
#plt.yscale("log")
plt.xticks(range(len(np.unique(labels["da_regions"].values))))
plt.show()

In [ ]:
plt.clf()
sns.displot(labels, x="da_regions", y='CD11b-PE', bins=12)

In [ ]:
#plt.clf()
#labels = labels[labels["da_regions"] != 0]
sns.displot(labels[labels["da_regions"] != 0] , x="da_regions", hue="sample", kind="hist", multiple="stack", bins=len(np.unique(labels["da_regions"].values)))
#plt.yscale("log")
plt.xticks(range(len(np.unique(labels["da_regions"].values))))
plt.show()

In [ ]:
regions = np.unique(labels["da_regions"].values)

genes = np.array(labels.columns[:-4])
de = np.ndarray((len(regions), len(genes))) 
    
for i, region in enumerate(regions):
    pvals = {}

    for gene in genes:
        x_cells = labels[labels["da_regions"] == region]
        y_cells = labels[labels["da_regions"] != region]
        x = x_cells[gene].values        
        y = y_cells[gene].values
        pvals[gene] = scipy.stats.mannwhitneyu(x,y).pvalue
    rejected = fdrcorrection(list(pvals.values()))
    degs_for_region = genes[np.where(rejected[0])]
    de[i] = rejected[0]

In [ ]:
pvals

In [ ]:
res = pd.DataFrame(de, columns=genes, index=regions, dtype=bool)

In [ ]:
res

In [ ]:
%pip install pydeseq2